In [ ]:
import copy
import numpy as np
import pandas as pd
import pyproj
from pyproj import CRS
import scipy
from scipy.spatial.distance import cdist
from sklearn.utils import check_random_state

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from night_horizons.container import DIContainer
from night_horizons.io_manager import IOManager
from night_horizons.raster import ReferencedImage, DatasetWrapper
from night_horizons.transformers.raster import RasterCoordinateTransformer
from night_horizons.data_io import GDALDatasetIO, TabularIO

## Setup


### Parameters


In [ ]:
local_options = {
    'assumed_crs': 'EPSG: 4326',
    'r_scaling': 1.,
    'padding': 1.,
    'search_range': 10000,
}

In [ ]:
container = DIContainer(
    config_filepath='./triangulation-exploration.yaml',
    local_options=local_options,
)

In [ ]:
container.register_service('io_manager', IOManager)
container.register_service(
    'crs',
    CRS,
    singleton=True,
)
container.register_service(
    'random_state',
    check_random_state,
    singleton=True,
)

### Code


In [ ]:
class Sensor:
    
    def __init__(self, x, y, r_scaling: float=container.config['r_scaling']):
        self.x = x
        self.y = y
        self.coords = (x, y)
        self.r_scaling = r_scaling
        
    def observe_sources(
        self,
        source_xs,
        source_ys,
        source_ls,
        theta_bins=np.linspace(-np.pi, np.pi, 64),
    ):
        
        # Center on source
        source_xs = copy.copy(source_xs - self.x)
        source_ys = copy.copy(source_ys - self.y)
        
        # Calculate angles
        thetas = np.arctan2(source_ys, source_xs)
        thetas = thetas.flatten()
        
        # Calculate brightness scalings
        rs = np.sqrt(source_xs**2. + source_ys**2.)
        surface_areas = np.pi * rs**self.r_scaling
        
        # Get weights
        weights = copy.copy(source_ls.flatten())
        surface_areas = surface_areas.flatten()
        weights[weights>0.] /= surface_areas[weights>0.]
        
        fluxs, theta_bins = np.histogram(
            thetas.flatten(),
            bins=theta_bins,
            weights=weights,
        )
        thetas = 0.5 * ( theta_bins[:-1] + theta_bins[1:] )
        
        return fluxs, thetas
    
    def map_from_observations(self, thetas, fluxs, xs, ys, kind='nearest'):
        
        xs = copy.copy(xs - self.x)
        ys = copy.copy(ys - self.y)

        thetas_per_coord = np.arctan2(ys, xs)
        above_bounds = thetas_per_coord > thetas.max()
        below_bounds = thetas_per_coord < thetas.min()
        in_bounds = np.invert(above_bounds) & np.invert(below_bounds)
        
        flux_mapping = scipy.interpolate.interp1d(thetas, fluxs, kind=kind)
        
        fluxs_per_coord = np.full(thetas_per_coord.shape, np.nan)
        fluxs_per_coord[above_bounds] = fluxs[-1]
        fluxs_per_coord[below_bounds] = fluxs[0]
        fluxs_per_coord[in_bounds] = flux_mapping(thetas_per_coord[in_bounds])

        rs = np.sqrt(xs**2. + ys**2.)
        surface_areas = np.pi * rs**2.

        allowable_fluxs = fluxs_per_coord * surface_areas

        return allowable_fluxs

In [ ]:
class SensorArray:
    
    def __init__(self, sensors):
        self.sensors = sensors
        
    def observe_sources(self, source_xs, source_ys, source_ls, theta_bins=np.linspace(-np.pi, np.pi, 128)):
        
        self.sensor_fluxs = []
        for i, sensor_i in enumerate(self.sensors):
            fluxs, thetas = sensor_i.observe_sources(source_xs, source_ys, source_ls, theta_bins)
            self.sensor_fluxs.append(fluxs)
        self.thetas = thetas
        
        return self.sensor_fluxs, self.thetas
        
    def plot_observations(self, ax, sensor_fluxs, thetas, **pcolor_kwargs):
        
        n_sensors = len(sensor_fluxs)
        if n_sensors == 1:
            fluxs = sensor_fluxs[0]
            tileshape = (2,1)
            flux_arr = np.tile(fluxs, tileshape)
            thetas_tiled = np.tile(thetas, tileshape)
            display_ys = np.array( [ [ 0., ] * fluxs.size, [ 1., ] * fluxs.size ] )
        else:
            flux_arr = np.array(sensor_fluxs)
            thetas_tiled = np.tile(thetas,(n_sensors,1))
            display_ys = np.tile(np.arange(n_sensors), (flux_arr.shape[1],1)).transpose()
            
        pcolor_kwargs_used = {
            'cmap': 'Greys_r',
        }
        pcolor_kwargs_used.update(pcolor_kwargs)
            
        ax.pcolormesh(
            -thetas_tiled,
            display_ys,
            flux_arr,
            **pcolor_kwargs_used
        )
        
    def estimate_allowed_luminosities_on_grid(self, grid_res=1024):
        
        self.xs_for_estimate = np.linspace(xs.min(), xs.max(), grid_res)
        self.ys_for_estimate = np.linspace(ys.min(), ys.max(), grid_res)
        xs_grid, ys_grid = np.meshgrid(self.xs_for_estimate, self.ys_for_estimate)
        
        self.allowed_luminosities = []
        for i, sensor_i in enumerate(s_arr.sensors):
            allowed_luminosities_i = sensor_i.map_from_observations(self.thetas, self.sensor_fluxs[i], xs_grid, ys_grid)
            self.allowed_luminosities.append(allowed_luminosities_i)
            
        return self.allowed_luminosities

# Simplified


### Parameters


In [ ]:
palette = sns.color_palette('deep')

In [ ]:
brightness_norm = matplotlib.colors.LogNorm(vmin=0.001, vmax=1., clip=True)

In [ ]:
# Generate source grid
source_grid_res = (16,16)
source_type = 'point'
n_lights = 5
rng = container.get_service('random_state')
light_luminosities = rng.uniform(1., 1., n_lights)

source_grid = np.zeros(source_grid_res)
xs = np.arange(source_grid.shape[0])
ys = np.arange(source_grid.shape[1])
x_lights = np.random.choice(xs, n_lights)
y_lights = np.random.choice(ys, n_lights)
for i in range(n_lights):
    
    if source_type == 'point':
        source_grid[x_lights[i],y_lights[i]] = light_luminosities[i]
    elif source_type == 'normal':
        x_pdf = scipy.stats.norm(x_lights[i]).pdf(xs)
        y_pdf = scipy.stats.norm(y_lights[i]).pdf(ys)
        x_pdf_mesh, y_pdf_mesh = np.meshgrid(x_pdf, y_pdf)
        source_grid += light_luminosities[i] * (x_pdf_mesh*y_pdf_mesh)

In [ ]:
# Generate sensors
n_sensors = 3

sensor_xs = np.random.choice(xs, n_sensors)
sensor_ys = np.random.choice(ys, n_sensors)
sensors = []
for i in range(n_sensors):
    sensors.append(Sensor(sensor_xs[i], sensor_ys[i]))
s_arr = SensorArray(sensors)

In [ ]:
# Have sensors view the sources
xs_grid, ys_grid = np.meshgrid(xs, ys)
sensor_fluxs, thetas = s_arr.observe_sources(xs_grid, ys_grid, source_grid)

## View Sources and Sensors


### View


In [ ]:
fig = plt.figure()
ax  = plt.gca()

ax.pcolormesh(
    xs,
    ys,
    source_grid,
    cmap = 'Greys_r',
    norm = brightness_norm,
)

for i, sensor_i in enumerate(s_arr.sensors):
    ax.scatter(
        sensor_i.x,
        sensor_i.y,
        color = palette[i],
        s = 100,
    )

ax.set_aspect('equal')

In [ ]:
fig = plt.figure(figsize=(10,len(s_arr.sensors)))
ax = plt.gca()

s_arr.plot_observations(ax, s_arr.sensor_fluxs, s_arr.thetas, norm=brightness_norm)

# X ticks
dx = 0.25
xtick_multiples = np.array([-1, -0.5, 0., 0.5, 1.])
xticks = xtick_multiples * np.pi
xtick_labels = [ '{:.2g}'.format( -_ ) + r'$\pi$' for _ in xtick_multiples ]
xticks = ax.set_xticks(xticks, xtick_labels)

# Y labels
ax.tick_params(left=False, labelleft=False)
for i, sensor_i in enumerate(s_arr.sensors):
    ax.annotate(
        'sensor {}'.format(i),
        xy = (-np.pi,i),
        xycoords = 'data',
        xytext = (-5,0),
        textcoords = 'offset points',
        color = palette[i],
        ha = 'right',
        fontsize = 14,
    )

## Recreate Light Source


In [ ]:
allowed_ls = s_arr.estimate_allowed_luminosities_on_grid()

In [ ]:
for i, allowed_l in enumerate(allowed_ls):
    
    fig = plt.figure()
    ax  = plt.gca()

    ax.pcolormesh(
        s_arr.xs_for_estimate,
        s_arr.ys_for_estimate,
        allowed_l,
        cmap = 'Greys_r',
        norm = matplotlib.colors.Normalize(0, 1),
    )

    ax.scatter(
        s_arr.sensors[i].x,
        s_arr.sensors[i].y,
        color = palette[i],
        s = 100,
    )

    ax.set_aspect('equal')

In [ ]:

fig = plt.figure()
ax  = plt.gca()

ax.pcolormesh(
    s_arr.xs_for_estimate,
    s_arr.ys_for_estimate,
    np.array(allowed_ls).sum(axis=0),
    cmap = 'Greys_r',
    norm = matplotlib.colors.Normalize(0, 1),
)
for i, allowed_l in enumerate(allowed_ls):

    ax.scatter(
        s_arr.sensors[i].x,
        s_arr.sensors[i].y,
        color = palette[i],
        s = 100,
    )

ax.set_aspect('equal')

# Real Data


In [ ]:
io_manager = container.get_service('io_manager')

## Get GDAL Dataset


In [ ]:
dataset = GDALDatasetIO.load_from_viirs_hdf5(
    io_manager.input_filepaths['viirs_raw_data']
)

In [ ]:
proj_str = dataset.GetProjection()
if proj_str != '':
    image_crs = pyproj.CRS(proj_str)
else:
    image_crs = pyproj.CRS(container.config['assumed_crs'])

In [ ]:
x_min, pixel_width, _, y_max, _, pixel_height = dataset.GetGeoTransform()

In [ ]:
x_max = x_min + pixel_width * dataset.RasterXSize
y_min = y_max + pixel_height * dataset.RasterYSize

## Get Camera Coords


In [ ]:
df = TabularIO.load(
    io_manager.input_filepaths['triangulation_metadata'],
)

In [ ]:
df['date'] = pd.to_datetime(
    df['created_at'].str.split('T').str[0],
    format='mixed'
)

In [ ]:
is_set_up = df['2_Are_you_deploying_'] == 'Deploying GONet'
is_right_date = df['date'] > pd.to_datetime('2023-05-01') 
has_cam_num = df['4_What_is_the_GONet_'].notna()
valid = is_set_up & is_right_date & has_cam_num

In [ ]:
lonlat = df.loc[
    valid,
    ['long_5_What_are_the_latit', 'lat_5_What_are_the_latit']
].values

In [ ]:
latlon_crs = pyproj.CRS('EPSG:4326')
crs = container.get_service('crs')

In [ ]:
latlon_to_cart = pyproj.Transformer.from_crs(
    latlon_crs,
    crs,
    always_xy=True
)

In [ ]:
latlon_to_image = pyproj.Transformer.from_crs(
    latlon_crs,
    image_crs,
    always_xy=True
)
image_to_latlon = pyproj.Transformer.from_crs(
    image_crs,
    latlon_crs,
    always_xy=True
)

In [ ]:
xs, ys = latlon_to_image.transform(lonlat[:,0], lonlat[:,1])

In [ ]:
plt.scatter(
    xs,
    ys,
)

In [ ]:
assert (xs >= x_min).all()
assert (xs <= x_max).all()
assert (ys >= y_min).all()
assert (ys <= y_max).all()

## Get Image


In [ ]:
hypotenuse = np.sqrt((xs.max() - xs.min())**2. + (ys.max() - ys.min())**2.)
padding = container.config['padding'] * hypotenuse
x_min = xs.min() - padding
x_max = xs.max() + padding
y_min = ys.min() - padding
y_max = ys.max() + padding

In [ ]:
tfer = RasterCoordinateTransformer()
tfer.fit_to_dataset(dataset)
x_off, y_off, x_size, y_size = tfer.physical_to_pixel(
    x_min=x_min,
    x_max=x_max,
    y_min=y_min,
    y_max=y_max
)

In [ ]:
img = dataset.ReadAsArray(
    int(x_off), int(y_off), int(x_size), int(y_size)
)
img = img / img.max()

In [ ]:
x_edges = np.linspace(x_min, x_max, img.shape[1])
y_edges = np.linspace(y_max, y_min, img.shape[0])

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = plt.gca()

ax.pcolormesh(
    x_edges,
    y_edges,
    img,
    cmap = 'Greys_r',
    norm=matplotlib.colors.LogNorm(),
)

ax.scatter(
    xs,
    ys,
    color='k',
)
for i, (x_i, y_i) in enumerate(zip(xs, ys)):
    ax.annotate(
        str(i),
        xy = (x_i, y_i),
        xycoords = 'data',
        xytext = (-1,-1),
        textcoords = 'offset points',
        color = palette[i],
        fontsize = 18,
        ha='right',
        va='top',
    )

ax.set_aspect('equal')

## Mock Observations


### Generate


In [ ]:
# Generate sensors
n_sensors = len(xs)
sensors = []
for i in range(n_sensors):
    sensors.append(Sensor(xs[i], ys[i]))
s_arr = SensorArray(sensors)

In [ ]:
# Have sensors view the sources
xs_grid, ys_grid = np.meshgrid(x_edges, y_edges)
sensor_fluxs, thetas = s_arr.observe_sources(xs_grid, ys_grid, img)

### Visualize


In [ ]:
fig = plt.figure(figsize=(10,len(s_arr.sensors)))
ax = plt.gca()

s_arr.plot_observations(ax, s_arr.sensor_fluxs, s_arr.thetas)

In [ ]:
subplot_mosaic = [[_, ] for _ in np.arange(len(s_arr.sensors))]

fig = plt.figure(figsize=(10, 2 * len(s_arr.sensors)))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

for i, sensor_i in enumerate(s_arr.sensors):
    ax = ax_dict[i]
    ax.step(
        s_arr.thetas[::-1],
        s_arr.sensor_fluxs[i], # / s_arr.sensor_fluxs[i].max(),
        color=palette[i],
    )

    ax.annotate(
        'sensor {}'.format(i),
        xy = (0.0, 1.0),
        xycoords = 'axes fraction',
        xytext = (5, -5),
        textcoords = 'offset points',
        color = palette[i],
        ha = 'left',
        va = 'top',
        fontsize = 14,
    )

    # ax.set_ylim(0, 150)
    # ax.set_yscale('log')

for ax_key, ax in ax_dict.items():

    # X ticks
    dx = 0.25
    xtick_multiples = np.array([-1, -0.5, 0., 0.5, 1.])
    xticks = xtick_multiples * np.pi
    xtick_labels = [ '{:.2g}'.format( -_ ) + r'$\pi$' for _ in xtick_multiples ]
    xticks = ax.set_xticks(xticks, xtick_labels)

    # Y labels
    ax.tick_params(left=False, labelleft=False)
    # for i, sensor_i in enumerate(s_arr.sensors):
    #     ax.annotate(
    #         'sensor {}'.format(i),
    #         xy = (-np.pi,i),
    #         xycoords = 'data',
    #         xytext = (-5,0),
    #         textcoords = 'offset points',
    #         color = palette[i],
    #         ha = 'right',
    #         fontsize = 14,
        #)

- Effective smoothing length is tiny (maybe blur)
- The host pixel of the sensor gets a huge boost (and all those very close)
  - The direction will also be quantized.
- Weird cloud masking if we go farther out.
- Show 1/r^2 weighted plots.
- Get conway data.
- Fit to data to get best r^-alpha scaling


In [ ]:
allowed_ls = s_arr.estimate_allowed_luminosities_on_grid()

In [ ]:
for i, allowed_l in enumerate(allowed_ls):
    
    fig = plt.figure()
    ax  = plt.gca()

    ax.pcolormesh(
        s_arr.xs_for_estimate,
        s_arr.ys_for_estimate,
        allowed_l,
        cmap = 'Greys_r',
        norm = matplotlib.colors.Normalize(0, 1),
    )

    ax.scatter(
        s_arr.sensors[i].x,
        s_arr.sensors[i].y,
        color = palette[i],
        s = 100,
    )

    ax.set_aspect('equal')

# Identifying Candidate Deployment Locations


In [ ]:
container.config['search_range'] = 0.5

In [ ]:
x_min = xs.min() - container.config['search_range']
x_max = xs.max() + container.config['search_range']
y_min = ys.min() - container.config['search_range']
y_max = ys.max() + container.config['search_range']

In [ ]:
tfer = RasterCoordinateTransformer()
tfer.fit_to_dataset(dataset)
x_off, y_off, x_size, y_size = tfer.physical_to_pixel(
    x_min=x_min,
    x_max=x_max,
    y_min=y_min,
    y_max=y_max
)

In [ ]:
img = dataset.ReadAsArray(
    int(x_off), int(y_off), int(x_size), int(y_size)
)

In [ ]:
x_edges = np.linspace(x_min, x_max, img.shape[1])
y_edges = np.linspace(y_max, y_min, img.shape[0])

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = plt.gca()

ax.pcolormesh(
    x_edges,
    y_edges,
    img,
    cmap = 'Greys_r',
    norm=matplotlib.colors.LogNorm(),
)

ax.scatter(
    xs,
    ys,
    color='k',
)
for i, (x_i, y_i) in enumerate(zip(xs, ys)):
    ax.annotate(
        str(i),
        xy = (x_i, y_i),
        xycoords = 'data',
        xytext = (-1,-1),
        textcoords = 'offset points',
        color = palette[i],
        fontsize = 18,
        ha='right',
        va='top',
    )

# ax.set_aspect('equal')

In [ ]:
import cv2
blur = cv2.GaussianBlur(img,(5,5),0)

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = plt.gca()

ax.pcolormesh(
    x_edges,
    y_edges,
    blur,
    cmap = 'viridis',
    norm=matplotlib.colors.Normalize(1e2, 3e2),
)

ax.scatter(
    xs,
    ys,
    color='k',
)
for i, (x_i, y_i) in enumerate(zip(xs, ys)):
    ax.annotate(
        str(i),
        xy = (x_i, y_i),
        xycoords = 'data',
        xytext = (-1,-1),
        textcoords = 'offset points',
        color = palette[i],
        fontsize = 18,
        ha='right',
        va='top',
    )

# ax.set_aspect('equal')

In [ ]:
dx = x_edges[1] - x_edges[0]
dy = y_edges[0] - y_edges[1] 


In [ ]:
x_min = xs.mean() - dx
x_max = xs.mean() + dx
y_min = ys.mean() - dy
y_max = ys.mean() + dy

In [ ]:
tfer = RasterCoordinateTransformer()
tfer.fit_to_dataset(dataset)
x_off, y_off, x_size, y_size = tfer.physical_to_pixel(
    x_min=x_min,
    x_max=x_max,
    y_min=y_min,
    y_max=y_max
)

In [ ]:
img_local = dataset.ReadAsArray(
    int(x_off), int(y_off), int(x_size), int(y_size)
)

In [ ]:
fig = plt.figure()
ax = plt.gca()

sns.histplot(
    blur.flatten(),
    ax = ax,
    log_scale=True,
)

ax.axvline(
    img_local.mean(),
    color='k',
)

# Two Questions for LP Research

## 1. How do humans affect the light levels on earth?

- What is the difference between local and non-local?
- Including what is the length scale at which we switch from local to non-local?
- In the model of $R(\theta) = R_{\rm local} + R_{\rm non-local}$,
  what is the contribution of $R_{\rm local}$ vs $R_{\rm non-local}$?
- Can we measure $R(\theta)$ at fixed $R_{\rm local}$, and thereby constrain $R_{\rm non-local}$?

## 2. How does nature respond?
